In [68]:
from ibots_db.schema import Data
from pydantic import BaseModel
from pathlib import Path
import shutil

In [6]:
Data.model_fields

{'people': FieldInfo(annotation=dict[str, Person], required=True),
 'orgs': FieldInfo(annotation=Orgs, required=True),
 'skills': FieldInfo(annotation=dict[str, Skill], required=True),
 'technologies': FieldInfo(annotation=dict[str, Technology], required=True),
 'images': FieldInfo(annotation=dict[str, Annotated[Path, PathType]], required=True)}

In [8]:
tech = Data.model_fields['technologies']

In [10]:
tech.annotation

dict[str, ibots_db.schema.Technology]

In [15]:
tech.get_default()

PydanticUndefined

In [18]:
tech.annotation

dict[str, ibots_db.schema.Technology]

In [24]:
from typing import get_args, get_origin

In [23]:
get_args(tech.annotation)

(str, ibots_db.schema.Technology)

In [25]:
get_origin(tech.annotation)

dict

In [40]:
get_origin(tech.annotation)

dict

---

In [ ]:
aa = {'a': 3}


In [110]:
from tempfile import TemporaryDirectory
from ibots_db.funs import DATA_PATH
from ibots_db.schema import Technology
import yaml

# assert key is an attribute of Data schema
key = 'technologies'
data = {'tech1': Technology(name='C-Sharp', icon='icon c-sharp', homepage='https://www.c-sharp.com')}

field = Data.model_fields[key]

# check that BaseModel given is the same type specified by the schema
assert get_origin(field.annotation) is dict
assert isinstance(data, dict)
key_type, value_type = get_args(field.annotation)
assert key_type is str
assert issubclass(value_type, BaseModel)
assert all(type(value) is value_type for key, value in data.items())

# Make yaml out of the data
yaml_data = {key + '.yaml': yaml.dump(value.model_dump(mode='json'), Dumper=yaml.Dumper) for key, value in data.items()}
for val in yaml_data.values():
    print(val)

# make a temporary directory and copy the db files into it.
temp_dir = Path(TemporaryDirectory(prefix='dbcheck').name)
print(temp_dir)
shutil.copytree(DATA_PATH, temp_dir, dirs_exist_ok=True)

if not temp_dir.joinpath(key).with_suffix('.yaml').exists():
    for k,value in yaml_data.items():
        temp_dir.joinpath(key).joinpath(k).write_text(value)

homepage: https://www.c-sharp.com/
icon: icon c-sharp
name: C-Sharp

C:\Users\nickdg\AppData\Local\Temp\dbcheckkjt3nh1u


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nickdg\\AppData\\Local\\Temp\\dbcheckkjt3nh1u\\technologies\\tech1.yaml'

---

In [49]:
issubclass(key_type, BaseModel)

False

In [94]:
BaseModel.model_dump?

Signature:
BaseModel.model_dump(
    self,
    *,
    mode: "Literal['json', 'python'] | str" = 'python',
    include: 'IncEx' = None,
    exclude: 'IncEx' = None,
    context: 'dict[str, Any] | None' = None,
    by_alias: 'bool' = False,
    exclude_unset: 'bool' = False,
    exclude_defaults: 'bool' = False,
    exclude_none: 'bool' = False,
    round_trip: 'bool' = False,
    warnings: "bool | Literal['none', 'warn', 'error']" = True,
    serialize_as_any: 'bool' = False,
) -> 'dict[str, Any]'
Docstring:
Usage docs: https://docs.pydantic.dev/2.7/concepts/serialization/#modelmodel_dump

Generate a dictionary representation of the model, optionally specifying which fields to include or exclude.

Args:
    mode: The mode in which `to_python` should run.
        If mode is 'json', the output will only contain JSON serializable types.
        If mode is 'python', the output may contain non-JSON-serializable Python objects.
    include: A set of fields to include in the output.
    exclud

In [60]:
BaseModel in value_type.mro()

True

In [50]:
value_type.mro()

[ibots_db.schema.Technology, pydantic.main.BaseModel, object]

In [57]:
class A: ...
class B(A): ...
class E(A): ...
class D(E): ...
class C(D, B): ...


In [52]:
A.mro()

[__main__.A, object]

In [53]:
B.mro()

[__main__.B, __main__.A, object]

In [54]:
D.mro()

[__main__.D, __main__.E, __main__.A, object]

In [58]:
C.mro()

[__main__.C, __main__.D, __main__.E, __main__.B, __main__.A, object]

In [56]:
type(C.mro())

list